In [1]:
import tensorflow as tf
# from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras import optimizers
from keras.models import load_model
from keras import backend as K

from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG

# Helper libraries
import numpy as np
import pandas as pd
from numpy import array
from numpy import argmax

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
class MachineLearning :
    def __init__(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train
        
    def train(self):
        self.sess = tf.Session()
        K.set_session(self.sess)
        
        self.__preprocessing()
        self.__model()
        self.model.fit(x=self.x_train, 
          y=self.y_train,
          validation_split=0.1,
          batch_size=2,
          shuffle=True,
          epochs=100)
        
        # saving model
        self.model.save("model_1.h5")
        
        # Use TF to save the graph model instead of Keras save model to load it in Golang
        builder = tf.saved_model.builder.SavedModelBuilder("myModel")
        # Tag the model, required for Go
        builder.add_meta_graph_and_variables(self.sess, ["myTag"])
        builder.save()
        self.sess.close()
        
    def load_model(self, class_file='classes.npy', model_file='model_1.h5'):
        with tf.Session() as sess:
            # load model
            self.model= load_model(model_file)

            # load class
            self.labels = np.load(class_file)
        
    def display_model(self):
        SVG(model_to_dot(self.model).create(prog='dot', format='svg'))
        
    def predict(self, data):
#         with tf.Session() as sess:
        print('All prediction of each class:',self.model.predict(np.array(data)))

        temp_prediction = self.model.predict_classes(np.array(data))
        predictions = []
        for i in temp_prediction:
            predictions.append(self.labels[i])

        print('Prediction class:', predictions)
        
    
    def __preprocessing(self):
        # convert x_train to numpy x_train
        self.x_train = np.array(self.x_train)
        
        # set label encoder for y_train
        label_encoder = LabelEncoder()
        encoded = label_encoder.fit_transform(self.y_train)
        
        # save class for the labels
        np.save('classes.npy', label_encoder.classes_)
        print(label_encoder.classes_)
        np.savetxt('test.txt', label_encoder.classes_, delimiter=',', fmt='%s')
        
        # binary encode
        onehot_encoder = OneHotEncoder(sparse=False)
        integer_encoded = encoded.reshape(len(encoded), 1)
        self.y_train = onehot_encoder.fit_transform(integer_encoded)
        
    def __model(self):
        self.model = Sequential([
            Flatten(input_shape=(4, 3), name="inputLayer"),
            Dense(32, activation='relu'),
            Dropout(0.3),
            Dense(64, activation='sigmoid'),
            Dropout(0.5),
            Dense(3, activation='softmax', name="inferenceLayer")
        ])
        self.model.compile(optimizer=optimizers.RMSprop(lr=0.001, rho=0.9),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        
        # print([n.name for n in tf.get_default_graph().as_graph_def().node])
        # SVG(model_to_dot(self.model).create(prog='dot', format='svg'))
        
if __name__ == '__main__':
    x_train = [
        [[1,1,1],[1,1,1],[1,1,1],[1,1,1]],
        [[2,2,2],[2,2,2],[2,2,2],[2,2,2]],
        [[3,3,3],[3,3,3],[3,3,3],[3,3,3]],
        [[4,4,4],[4,4,4],[4,4,4],[4,4,4]],
        [[5,5,5],[5,5,5],[5,5,5],[5,5,5]],
        [[6,6,6],[6,6,6],[6,6,6],[6,6,6]],
        [[7,7,7],[7,7,7],[7,7,7],[7,7,7]],
        [[8,8,8],[8,8,8],[8,8,8],[8,8,8]],
        [[9,9,9],[9,9,9],[9,9,9],[9,9,9]],
        [[10,10,10],[10,10,10],[10,10,10],[10,10,10]],
        [[11,11,11],[11,11,11],[11,11,11],[11,11,11]],
        [[22,22,22],[22,22,22],[22,22,22],[22,22,22]],
        [[33,33,33],[33,33,33],[33,33,33],[33,33,33]],
        [[44,44,44],[44,44,44],[44,44,44],[44,44,44]],
        [[55,55,55],[55,55,55],[55,55,55],[55,55,55]],
        [[-1,-1,-1],[-1,-1,-1],[-1,-1,-1],[-1,-1,-1]],
        [[-2,-2,-2],[-2,-2,-2],[-2,-2,-2],[-2,-2,-2]],
        [[-3,-3,-3],[-3,-3,-3],[-3,-3,-3],[-3,-3,-3]],
        [[-11,-11,-11],[-11,-11,-11],[-11,-11,-11],[-11,-11,-11]],
        [[-12,-12,-12],[-12,-12,-12],[-12,-12,-12],[-12,-12,-12]],
        [[-13,-13,-13],[-13,-13,-13],[-13,-13,-13],[-13,-13,-13]]
    
    ]
    
    y_train = [
        'satuan','satuan','satuan','satuan','satuan','satuan','satuan','satuan','satuan',
        'puluhan','puluhan','puluhan','puluhan','puluhan','puluhan',
        'minus','minus','minus','minus','minus','minus'
    ]
    
    
    x = MachineLearning(x_train, y_train)
    x.train()
#     x.load_model()

/home/nakama/anaconda3/envs/machine-learning/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
W0114 16:49:25.005710 140057503045440 deprecation_wrapper.py:119] From /home/nakama/anaconda3/envs/machine-learning/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0114 16:49:25.006891 140057503045440 deprecation_wrapper.py:119] From /home/nakama/anaconda3/envs/machin

['minus' 'puluhan' 'satuan']


W0114 16:49:25.265490 140057503045440 deprecation.py:323] From /home/nakama/anaconda3/envs/machine-learning/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 18 samples, validate on 3 samples
Epoch 1/100
18/18 [==============================] - 1s 51ms/step - loss: 1.7535 - acc: 0.3333 - val_loss: 0.1225 - val_acc: 1.0000
Epoch 2/100
18/18 [==============================] - 0s 2ms/step - loss: 1.3808 - acc: 0.3889 - val_loss: 0.0950 - val_acc: 1.0000
Epoch 3/100
18/18 [==============================] - 0s 1ms/step - loss: 1.4746 - acc: 0.3889 - val_loss: 0.0610 - val_acc: 1.0000
Epoch 4/100
18/18 [==============================] - 0s 2ms/step - loss: 1.0439 - acc: 0.5000 - val_loss: 0.0458 - val_acc: 1.0000
Epoch 5/100
18/18 [==============================] - 0s 2ms/step - loss: 0.9331 - acc: 0.5000 - val_loss: 0.0397 - val_acc: 1.0000
Epoch 6/100
18/18 [==============================] - 0s 2ms/step - loss: 0.7807 - acc: 0.6111 - val_loss: 0.0404 - val_acc: 1.0000
Epoch 7/100
18/18 [==============================] - 0s 1ms/step - loss: 0.7321 - acc: 0.5556 - val_loss: 0.0311 - val_acc: 1.0000
Epoch 8/100
18/18 [====================

Epoch 63/100
18/18 [==============================] - 0s 1ms/step - loss: 0.5611 - acc: 0.7222 - val_loss: 0.0043 - val_acc: 1.0000
Epoch 64/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5363 - acc: 0.7222 - val_loss: 0.0041 - val_acc: 1.0000
Epoch 65/100
18/18 [==============================] - 0s 2ms/step - loss: 0.3579 - acc: 0.8333 - val_loss: 0.0041 - val_acc: 1.0000
Epoch 66/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5170 - acc: 0.7778 - val_loss: 0.0041 - val_acc: 1.0000
Epoch 67/100
18/18 [==============================] - 0s 2ms/step - loss: 0.4099 - acc: 0.7778 - val_loss: 0.0040 - val_acc: 1.0000
Epoch 68/100
18/18 [==============================] - 0s 2ms/step - loss: 0.3491 - acc: 0.8333 - val_loss: 0.0040 - val_acc: 1.0000
Epoch 69/100
18/18 [==============================] - 0s 2ms/step - loss: 0.4609 - acc: 0.7778 - val_loss: 0.0040 - val_acc: 1.0000
Epoch 70/100
18/18 [==============================] - 0s 1ms/step - loss: 0.

In [3]:
# with tf.Session() as sess:
#     x.load_model()
with tf.Session() as sess:
    x.predict([[[-1,-2,-3],[-1,-2,-3],[1,2,3],[1,2,0]]])

FailedPreconditionError: Attempting to use uninitialized value dense_1/bias
	 [[{{node dense_1/bias/read}}]]

In [12]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

['inputLayer_input',
 'inputLayer/Shape',
 'inputLayer/strided_slice/stack',
 'inputLayer/strided_slice/stack_1',
 'inputLayer/strided_slice/stack_2',
 'inputLayer/strided_slice',
 'inputLayer/Const',
 'inputLayer/Prod',
 'inputLayer/stack/0',
 'inputLayer/stack',
 'inputLayer/Reshape',
 'dense_1/random_uniform/shape',
 'dense_1/random_uniform/min',
 'dense_1/random_uniform/max',
 'dense_1/random_uniform/RandomUniform',
 'dense_1/random_uniform/sub',
 'dense_1/random_uniform/mul',
 'dense_1/random_uniform',
 'dense_1/kernel',
 'dense_1/kernel/Assign',
 'dense_1/kernel/read',
 'dense_1/Const',
 'dense_1/bias',
 'dense_1/bias/Assign',
 'dense_1/bias/read',
 'dense_1/MatMul',
 'dense_1/BiasAdd',
 'dense_1/Relu',
 'dropout_1/keras_learning_phase/input',
 'dropout_1/keras_learning_phase',
 'dropout_1/cond/Switch',
 'dropout_1/cond/switch_t',
 'dropout_1/cond/switch_f',
 'dropout_1/cond/pred_id',
 'dropout_1/cond/mul/y',
 'dropout_1/cond/mul',
 'dropout_1/cond/mul/Switch',
 'dropout_1/cond/d